In [2]:
import torch
import dgl
import torch.nn as nn
import torch.optim as optim
from dgl.nn import GraphConv
import numpy as np

# 假设时间序列数据已经转化为图结构
# 创建一个简单的图:假设有10个时间点,每个时间点的数据是一个节点,节点之间的边表示时间步依赖
num_nodes = 10  # 假设有10个时间点
edges = [(i, i+1) for i in range(num_nodes-1)]  # 创建连接每个相邻时间点的边

# 创建DGL图
g = dgl.graph(edges)
g = g.add_nodes(num_nodes)

# 每个节点的特征是对应时间点的数据,假设时间序列数据是一个随机生成的数组
node_features = torch.randn(num_nodes, 5)  # 假设每个节点的特征是5维
g.ndata['feat'] = node_features

# 定义图神经网络模型
class GNN_TS_Model(nn.Module):
    def __init__(self, in_feats, hidden_size, out_feats):
        super(GNN_TS_Model, self).__init__()
        self.conv1 = GraphConv(in_feats, hidden_size)
        self.conv2 = GraphConv(hidden_size, out_feats)
        self.fc = nn.Linear(out_feats, 1)  # 预测一个标量值

    def forward(self, g):
        h = g.ndata['feat']
        h = torch.relu(self.conv1(g, h))  # 图卷积
        h = torch.relu(self.conv2(g, h))  # 第二层图卷积
        h = h.mean(dim=1)  # 聚合节点特征
        out = self.fc(h)  # 全连接层预测
        return out

# 初始化模型
model = GNN_TS_Model(in_feats=5, hidden_size=16, out_feats=8)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# 模拟训练过程
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # 前向传播
    output = model(g)
    
    # 假设目标是时间序列的最后一个时间点的值
    target = torch.randn(1)  # 目标值是随机生成的
    loss = criterion(output, target)
    
    # 反向传播
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


FileNotFoundError: Cannot find DGL C++ graphbolt library at d:\ProgramData\anaconda3\envs\pytorch\lib\site-packages\dgl\graphbolt\graphbolt_pytorch_2.4.1.dll